In [4]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer

class BertCNNBiLSTM(nn.Module):
    def __init__(self, bert_model_name='bert-base-uncased', num_classes=3, lstm_hidden_dim=256, cnn_out_channels=128, dropout_rate=0.5):
        super(BertCNNBiLSTM, self).__init__()

        self.bert = BertModel.from_pretrained(bert_model_name)
        self.hidden_size = self.bert.config.hidden_size

        self.conv1 = nn.Conv1d(in_channels=self.hidden_size, out_channels=cnn_out_channels, kernel_size=2)
        self.conv2 = nn.Conv1d(in_channels=self.hidden_size, out_channels=cnn_out_channels, kernel_size=3)
        self.conv3 = nn.Conv1d(in_channels=self.hidden_size, out_channels=cnn_out_channels, kernel_size=5)

        self.bilstm = nn.LSTM(input_size=cnn_out_channels * 3, hidden_size=lstm_hidden_dim, bidirectional=True, batch_first=True)

        self.fc = nn.Linear(lstm_hidden_dim * 2, num_classes)  # BiLSTM outputs hidden_dim * 2

        self.dropout = nn.Dropout(dropout_rate)

        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state

        x = last_hidden_state.permute(0, 2, 1)

        x1 = torch.relu(self.conv1(x))
        x2 = torch.relu(self.conv2(x))
        x3 = torch.relu(self.conv3(x))

        x1 = torch.max(x1, dim=2).values
        x2 = torch.max(x2, dim=2).values
        x3 = torch.max(x3, dim=2).values

        cnn_features = torch.cat((x1, x2, x3), dim=1)

        lstm_input = cnn_features.unsqueeze(1)

        lstm_out, _ = self.bilstm(lstm_input)
        lstm_out = lstm_out.squeeze(1)

        lstm_out = self.dropout(lstm_out)


        logits = self.fc(lstm_out)

        return self.softmax(logits)


In [12]:
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import numpy as np

class PoliticalDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = self.texts[item]
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long)
        }

def calculate_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return accuracy, precision, recall, f1

def train_model(model, train_dataloader, val_dataloader, optimizer, loss_fn, num_epochs=4, device='cuda'):
    model = model.to(device)
    best_f1 = 0
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        all_preds = []
        all_labels = []

        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            print(f"we have a batch with {len(input_ids)} elements")

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs

            loss = loss_fn(logits, labels)
            total_loss += loss.item()

            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_train_loss = total_loss / len(train_dataloader)
        accuracy, precision, recall, f1 = calculate_metrics(all_preds, all_labels)

        print(f"Epoch {epoch+1}:")
        print(f"Train Loss: {avg_train_loss:.4f}")
        print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

        model.eval()
        val_preds = []
        val_labels = []

        with torch.no_grad():
            for batch in val_dataloader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs

                preds = torch.argmax(logits, dim=1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        accuracy, precision, recall, f1 = calculate_metrics(val_preds, val_labels)
        print(f"Validation Metrics - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

    return model

df = pd.read_csv('all_years_merged.csv')
# df = df.sample(n=100, random_state=42)

label_map = {'left': 0, 'right': 1, 'center': 2}
df['Leaning'] = df['Leaning'].map(label_map)

batch_size = 32

train_texts, val_texts, train_labels, val_labels = train_test_split(df['Text'].tolist(), df['Leaning'].tolist(), test_size=0.1)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_dataset = PoliticalDataset(train_texts, train_labels, tokenizer, max_len=128)
val_dataset = PoliticalDataset(val_texts, val_labels, tokenizer, max_len=128)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

model = BertCNNBiLSTM(num_classes=3)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-5)

train_model(model, train_dataloader, val_dataloader, optimizer, loss_fn, num_epochs=4, device='cuda')



Epoch 1/4:   0%|          | 0/27 [00:00<?, ?it/s]

we have a batch with 32 elements


Epoch 1/4:   4%|▎         | 1/27 [00:00<00:09,  2.62it/s]

we have a batch with 32 elements


Epoch 1/4:   7%|▋         | 2/27 [00:01<00:13,  1.91it/s]

...

we have a batch with 32 elements


Epoch 4/4: 100%|██████████| 27/27 [00:17<00:00,  1.54it/s]

we have a batch with 21 elements
Epoch 4:
Train Loss: 0.7162
Accuracy: 0.9039, Precision: 0.9046, Recall: 0.9039, F1: 0.9041


Validation Metrics - Accuracy: 0.9053, Precision: 0.9079, Recall: 0.9053, F1: 0.9054


BertCNNBiLSTM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise

In [13]:
print(df['Leaning'].unique())


[0 1 2]


In [14]:
import torch
from transformers import BertTokenizer
from torch.nn import functional as F

def classify_text(input_text, model, device='cuda'):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    model.eval()

    encoding = tokenizer.encode_plus(
        input_text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs

        preds = torch.argmax(logits, dim=1).item()

    label_map_inv = {0: 'left', 1: 'right', 2: 'center'}
    predicted_label = label_map_inv[preds]

    return predicted_label

In [15]:
texts = [
    "Trump is the best",
    "Biden is the worst",
    "The Republican Party stands for freedom and capitalism.",
    "Democrats are champions of equality and social justice.",
    "Progressive policies are the way forward for America.",
    "Right-wing extremists are the biggest threat to democracy.",
    "Neither Democrats nor Republicans have all the answers.",
    "It's best to vote for the Republican candidate, as he will conserve America's values."
]

predicted_leanings = [classify_text(text, model) for text in texts]

for text, leaning in zip(texts, predicted_leanings):
    print(f"Text: {text}\nPredicted Leaning: {leaning}\n")


Text: Trump is the best
Predicted Leaning: right

Text: Biden is the worst
Predicted Leaning: right

Text: The Republican Party stands for freedom and capitalism.
Predicted Leaning: left

Text: Democrats are champions of equality and social justice.
Predicted Leaning: left

Text: Progressive policies are the way forward for America.
Predicted Leaning: left

Text: Right-wing extremists are the biggest threat to democracy.
Predicted Leaning: left

Text: Neither Democrats nor Republicans have all the answers.
Predicted Leaning: left

Text: It's best to vote for the Republican candidate, as he will conserve America's values.
Predicted Leaning: right

